# Example Whole-File Analysis

The following imports the text of the Rāmāyaṇa, which has been prepared ahead of time to have one verse per line (with verses consisting of three and four halves broken into two), for a total of 19,354 verses.

In [1]:
from tqdm import tqdm # for progress bar, pip install if desired, otherwise remove from for-loop below
from datetime import datetime, date

In [2]:
input_fn = 'R_cleaned.txt'
output_fn = input_fn[:input_fn.find('.')] + '_results' + input_fn[input_fn.find('.'):]

In [3]:
with open('%s' % input_fn, 'r') as input_f:
    input_data = input_f.read()
verses = input_data.split('\n')
print(len(verses))

19354


Here **skrutable**'s `MeterIdentifier` object is imported and instantiated.

In [4]:
from skrutable.meter_identification import MeterIdentifier
MI = MeterIdentifier()

Then, the verses are fed one at a time to the `MeterIdentifier`. To acheive maximum speed while maintaining accuracy, we can take advantage of pre-existing expert annotation. That is, some verses (generally samavṛtta) are already marked for all pāda breaks (ab ';', bc '/',  cd ';') and do not need to be resplit. On the other hand, other verses (generally anuṣṭubh) for which only the half-way point is marked (bc '/') must be resplit to find the exact location of the remaining breaks (in case of also, e.g., jāti verses, or because some verses may be hypo- or hypermetric). See below for more on dealing most efficiently with this annotation.

In [5]:
with open('%s' % output_fn, 'w') as output_f:

    starting_time = datetime.now().time()
    for v in tqdm(verses):
        v_content, v_label = v[:v.find('// ')+3], v[v.find('// ')+3:] # verse label is e.g. "1.001.001"
        if v_content.count(";") == 2:
            resplit_option = 'none'
        else:
            resplit_option = 'resplit_max'
        result = MI.identify_meter( v_content, resplit_option=resplit_option, from_scheme='IAST')
        # result = MI.identify_meter( v_label, resplit_option=resplit_option, from_scheme='IAST') # or this
        # result = MI.identify_meter( v, resplit_option=resplit_option, from_scheme='IAST') # or this
        output_f.write( v + '\t' + result.meter_label + '\n')
        # output_f.write( v + '\n\n' + result.summarize() + '\n') # or this

    ending_time = datetime.now().time()
    delta = datetime.combine(date.today(), ending_time) - datetime.combine(date.today(), starting_time)
    duration_secs = delta.seconds + delta.microseconds / 1000000
    output_f.write( "samāptam: %d padyāni, %f kṣaṇāḥ" % ( len(verses), duration_secs ) )

100%|██████████| 19354/19354 [00:02<00:00, 7387.01it/s]


## discussion

The total time required to ascertain the meter of all 19,354 verses depends on how many resplits are performed, and also on the machine used (here: MacBook Pro 2020 with 2 GHz Quad-Core Intel Core i5 processor). The speed baseline of about 3 seconds is given by performing no resplits and attempting to identify each verse only once. However, any verses without four explicit pāda breaks, e.g. anuṣṭubhs that mark only the midpoint, will fail to be identified. 

Better accuracy is provided by the two auto-re-split options, `resplit_max` and `resplit_lite`, both of which begin by breaking the verse input into even or near-even quarters based on overall syllable count. They then proceed to progressively flip groups of syllables over the provisional pāda boundaries, attempting a new identification for each new configuration, and scoring and recording all identification results that emerge. Finally, for each verse, the highest-scoring successful identification is chosen as the result. 

Of the two available options, `resplit_max` extends this treatment approximately half the length of the initialized pādas (i.e., as far as possible until overlaps would occur). This setting is computationally least efficient but acheives much better results, especially on anuṣṭubhs, which can then basically all be identified correctly. However, this flat resplitting treatment does not take into account the useful annotation in the samavṛtta verses, and some loss of accuracy occurs there. The total time, even with now nearly a million identification attempts rather than just twenty thousand, is still only 93 seconds.

To solve the above-mentioned accuracy problem while also reducing inefficiency, the `resplit_lite` option involves the same overall resplit treatment but extends it only half as far in either direction from the initialized pāda breaks, and it also includes a further option (controlled in `config.json`) to retain a fixed midpoint throughout the resplit process, since one can generally be very confident about the bc pāda break (marked by '/'). In this way, far fewer resplits need be performed in order to acheieve (in the case of anuṣṭubhs) the same or (in the case of samavṛttas) much better results. The total time is reduced to about 6 seconds.

A further small gain can be had by explicitly looking for whether all four pāda breaks are already provided (e.g., when `v_content.count(";") == 2`) and in such cases forgoing resplit treatment altogether with `resplit_option='none'`, and otherwise using `resplit_lite`. Accuracy is slightly improved yet again, and the total computation time comes down to about 5 seconds for all 19,354 verses.

This means that nearly all metrical texts, generally much smaller than the Rāmāyaṇa, can be analyzed in just a few fractions of a second, whereas the Mahābhārata would be on the order of 1 minute of processing time.

And of course, since **skrutable** has found the correct pāda breaks in most cases, it would be relatively easy to output a new version of the text with all pāda breaks explicitly marked. Such a further annotated text could then be run again through the flat `resplit_option='none'` baseline procedure, now with the benefit of enough additional information to not only acheive full accuracy but do so in the least possible time, if one wanted to.

More important, however, would be opportunity for an expert to intervene and correct those particular textual errors that could be revealed through the above process, thereby leading to an improvement of the content of the text itself.